In [9]:
import os, sys, email
import numpy as np 
import pandas as pd
import random

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# mv /Users/leonyin/Downloads/emails.csv data/

In [2]:
emails_df = pd.read_csv('data/emails.csv')

In [6]:
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [10]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [11]:
len(emails_df)

517401

In [13]:
emails_df.to_csv('data/emails_clean.tsv.gz', index=False, sep='\t', compression='gzip')

In [119]:
emails_df['To'].iloc[0]

frozenset({'tim.belden@enron.com'})

In [22]:
gold_df = emails_df[emails_df['content'].str.contains('golf', case=False)]

In [31]:
print(gold_df['content'].iloc[5])

[IMAGE]
Yahoo! sent this email to you because your Yahoo! Account Information =20
indicated that you wish to receive special offers. If you do not want to=20
receive further mailings  from Yahoo! Delivers, unsubscribe now by  clickin=
g=20
here. You are subscribed at: pallen@ect.enron.com

masthead=09 Yahoo!=20
=09
=09



May 2001


Greetings! Here's a look at some of the things happening on Yahoo! in May:







New Features & Services


[IMAGE]
Find Last Minute Mother's Day Gifts  -  Don't panic if you haven't found th=
e=20
perfect gift for Mom. Visit the Last Minute Mother's Day Gift Center on=20
Yahoo! Shopping. You'll find outstanding merchants and guaranteed delivery =
in=20
time for Mom's special day.
[IMAGE]
Got Stuff to Sell? It's a Great Time to Try Auctions  - Every time you=20
successfully sell an item on Yahoo! Auctions between now and June 4, you'll=
=20
be entered in the Yahoo! Auctions $2 Million Sweepstakes for a chance to wi=
n=20
one of twenty $100,000 prize package

In [116]:
emails_df[emails_df['To'].str.contains('enron', case=False)]

KeyError: '[ nan  nan  nan ...,  nan  nan  nan] not in index'

In [12]:
import spacy

In [13]:
nlp = spacy.load('en')

In [14]:
print(emails_df['content'].iloc[973])

Reagan,

Thank you for the quick response on the bid for the residence.  Below is a 
list of questions on the specs:

1.  Is the framing Lumber #2 yellow pine?  Wouldn't fir or spruce warp less 
and cost about the same?

2.  What type of floor joist would be used?  2x12 or some sort of factory 
joist?

3.  What type for roof framing?  On site built rafters? or engineered trusses?

4.  Are you planning for insulation between floors to dampen sound?  What 
type of insulation in floors and ceiling?  Batts or blown?  Fiberglass or 
Cellulose?

5.  Any ridge venting or other vents (power or turbine)?

6. Did you bid for interior windows to have trim on 4 sides?  I didn't know 
the difference between an apron and a stool.

7.  Do you do anything special under the upstairs tile floors to prevent 
cracking?  Double plywood or hardi board underlay?

8.  On the stairs, did you allow for a bannister?  I was thinking a partial 
one out of iron.  Only about 5 feet.

9.  I did not label it on the pl

In [15]:
doc = nlp(emails_df['content'].iloc[3].decode(encoding='utf-8', errors='strict'))

In [19]:
doc

Randy,

 Can you send me a schedule of the salary and level of everyone in the 
scheduling group.  Plus your thoughts on any changes that need to be made.  
(Patti S for example)

Phillip

In [16]:
doc.ents

(Randy, Patti S, Phillip)

In [17]:
[_.pos_ for _ in doc]

[u'PROPN',
 u'PUNCT',
 u'SPACE',
 u'VERB',
 u'PRON',
 u'VERB',
 u'PRON',
 u'DET',
 u'NOUN',
 u'ADP',
 u'DET',
 u'NOUN',
 u'CCONJ',
 u'NOUN',
 u'ADP',
 u'NOUN',
 u'ADP',
 u'DET',
 u'SPACE',
 u'NOUN',
 u'NOUN',
 u'PUNCT',
 u'SPACE',
 u'CCONJ',
 u'ADJ',
 u'NOUN',
 u'ADP',
 u'DET',
 u'NOUN',
 u'ADJ',
 u'VERB',
 u'PART',
 u'VERB',
 u'VERB',
 u'PUNCT',
 u'SPACE',
 u'PUNCT',
 u'PROPN',
 u'PROPN',
 u'ADP',
 u'NOUN',
 u'PUNCT',
 u'SPACE',
 u'PROPN']

In [20]:
[_.ent_type_ for _ in doc]

[u'PERSON',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'PERSON',
 u'PERSON',
 u'',
 u'',
 u'',
 u'',
 u'PERSON']

In [85]:
[_.tag_ for _ in nlp(u"She is speaking.")]

[u'PRP', u'VBZ', u'VBG', u'.']

In [88]:
[_.tag_ for _ in nlp(u"The backpack is his.")]

[u'DT', u'NN', u'VBZ', u'PRP$', u'.']

In [89]:
gender_neutral = {
    'PRF' : 'Hirself/Zirself',
    'WP$': 'Hirs/Zirs',
    'PRP$' : 'Hirs/Zirs',
    'PRP' : 'Ze'
}

In [76]:
for word in doc:
    print(word.text, word.tag_)

(u'Randy', u'NNP')
(u',', u',')
(u'\n\n ', u'SP')
(u'Can', u'MD')
(u'you', u'PRP')
(u'send', u'VB')
(u'me', u'PRP')
(u'a', u'DT')
(u'schedule', u'NN')
(u'of', u'IN')
(u'the', u'DT')
(u'salary', u'NN')
(u'and', u'CC')
(u'level', u'NN')
(u'of', u'IN')
(u'everyone', u'NN')
(u'in', u'IN')
(u'the', u'DT')
(u'\n', u'SP')
(u'scheduling', u'NN')
(u'group', u'NN')
(u'.', u'.')
(u' ', u'SP')
(u'Plus', u'CC')
(u'your', u'PRP$')
(u'thoughts', u'NNS')
(u'on', u'IN')
(u'any', u'DT')
(u'changes', u'NNS')
(u'that', u'WDT')
(u'need', u'VBP')
(u'to', u'TO')
(u'be', u'VB')
(u'made', u'VBN')
(u'.', u'.')
(u' \n', u'SP')
(u'(', u'-LRB-')
(u'Patti', u'NNP')
(u'S', u'NNP')
(u'for', u'IN')
(u'example', u'NN')
(u')', u'-RRB-')
(u'\n\n', u'SP')
(u'Phillip', u'NNP')


In [75]:
for word in doc:
    print(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_)

(u'Randy', 283848, u'randy', 475, u'NNP', 94, u'PROPN')
(u',', 450, u',', 450, u',', 95, u'PUNCT')
(u'\n\n ', 2446, u'\n\n ', 485, u'SP', 101, u'SPACE')
(u'Can', 586, u'can', 471, u'MD', 98, u'VERB')
(u'you', 757862, u'-PRON-', 479, u'PRP', 93, u'PRON')
(u'send', 1919, u'send', 488, u'VB', 98, u'VERB')
(u'me', 757862, u'-PRON-', 479, u'PRP', 93, u'PRON')
(u'a', 506, u'a', 460, u'DT', 88, u'DET')
(u'schedule', 7517, u'schedule', 474, u'NN', 90, u'NOUN')
(u'of', 510, u'of', 466, u'IN', 83, u'ADP')
(u'the', 501, u'the', 460, u'DT', 88, u'DET')
(u'salary', 4289, u'salary', 474, u'NN', 90, u'NOUN')
(u'and', 512, u'and', 458, u'CC', 87, u'CCONJ')
(u'level', 1203, u'level', 474, u'NN', 90, u'NOUN')
(u'of', 510, u'of', 466, u'IN', 83, u'ADP')
(u'everyone', 930, u'everyone', 474, u'NN', 90, u'NOUN')
(u'in', 522, u'in', 466, u'IN', 83, u'ADP')
(u'the', 501, u'the', 460, u'DT', 88, u'DET')
(u'\n', 518, u'\n', 485, u'SP', 101, u'SPACE')
(u'scheduling', 92101, u'scheduling', 474, u'NN', 90, u'NOUN'

In [54]:
propn_idx = [i for i, _ in enumerate(doc) if _.pos_ == u'PROPN']

In [71]:
people_names = [
    'Brooklyn',
    'Chastity',
    'Lil B',
    'Pauly'
]

In [ ]:
common_fuck_ups = {
    'girl' : 'person',
    'girls' : 'people',
    'boy' : 'child',
    'boys' : 'children',
    'man': 'he/his/'
}

In [ ]:
for word in doc:
    print(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_)

In [92]:
for i, _ in enumerate(txt):
    if i in propn_idx:
        print(random.choice(people_names))
    else:
        print(_)

Pauly
,


 
Can
you
send
me
a
schedule
of
the
salary
and
level
of
everyone
in
the


scheduling
group
.
 
Plus
your
thoughts
on
any
changes
that
need
to
be
made
.
 

(
Lil B
Pauly
for
example
)



Pauly
